# Tutorial 6 - Summarising and interpreting spatiotemporal data 

This tutorial demonstrates how to combine a polygon vector file with regions 
to calculate regional statistics, and how to save them and map them. 

More generally, we will see one commonly followed example of how you can combine the strengths 
of GIS software (in this case AcrGIS) with those of matrix processing software 
(in this case MatLab).

This tutorial covers the following technical concepts:

* Calculating temporal statistics
* Temporal aggregation
* Combining a polygon vector file with gridded data
* Creating a query grid 
* Reading a shapefile and its attributes
* Calculating spatial statistics
* Manipulating and sorting tables
* Exporting to csv and opening in Excel or GIS
* Changing the layout of a scatterplot and exploring data

As an example topic, we will look at the variability in the availability of 
surface water resources globally. The Bureau of Meteorology (BoM) frequently 
uses a graph to argue that Australia's water resources are very unpredictable 
compared to those in other countries. (And, by implication, that high-quality 
water information as provided by BoM is necessary).

![](data/bom-slide.jpg)

The graph above was based on measured streamflow data from (typically small) 
catchments. Such measurements are only available for the countries listed, and 
in many cases rely on a very small number of catchments. Instead, we now have 
global streamflow estimates from modelling that allow us to recreate the information 
shown above, for all of the world's surface and countries. In this 
tutorial example, we will do the following steps:

* Summarise the global streamflow timeseries into grids with annual values.
* Calculate the temporal statistics mean and standard deviation, and from 
  that, the coefficient of variation.
* In ArcGIS, use a global polygon vector map of countries to produce a query 
  grid that we can use to calculate regional statistics.
* Caculate statistics by country.
* Make a ranking similar to the slide above.
* Export the tabulated results as a csv file, so we might make a graph in 
  Excel or a thematic map in GIS.

We will be using gridded streamflow estimated using the W3 version 2.0 
model-data fusion system developed at ANU. The data were produced as part of 
an international Europe-led project called eartH2Observe 
([this website has details](http://www.earth2observe.eu) if you are interested). Other data available from 
the same system include, for example, global soil moisture, snowpack, vegetation 
carbon uptake, albedo, radiation and energy fluxes, and so on. You can see a 
list of variables 
[via this link,](http://dap.nci.org.au/thredds/remoteCatalogService?catalog=http://dapds00.nci.org.au/thredds/catalog/ub8/global/W3/e2o/wrr2/catalog.xml)
and could use any of them in your research. To reduce the volume 
of data we need to process, here we will use estimates averaged to monthly time 
step and 0.25 degrees, from the original daily time step and 0.05 degrees.

In [ ]:
# As usual, we start with our imports
import xarray as xr
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
seaborn.set_style('dark')

## Loading "Big Data" in chunks

"Big Data" is different sizes to different people.  A technical definition is that 
'small data fits in memory (< 8GB), medium data fits on disk (8GB - 4TB), big data doesn't (4TB+)'...
but more common is the personalised definition: 'big data is anything too large for your tools'.
In today's tutorial we'll see how to work effectively with anything that fits on a
personal computer, using tools that scale up to a supercomputer.
(with a small dataset as example, so our downloads avoid knocking over the network)

We'll start by taking a peek into the dataset, simply by opening it.  This is much
easier than in Matlab because Xarray does not load the data until you use it, 
only the metadata and coordinates.  When you *do* use it though, the whole variable is 
loaded at once - unless you tell Xarray to "chunk" your data (more on that later).

In [ ]:
url = 'http://dapds00.nci.org.au/thredds/dodsC/ub8/global/W3/e2o/wrr2/e2o_anu_wrr2_glob15_mon_Runoff_2000-2014.nc'
# Not chunked, but we need to inspect it to determine the right chunk size
unchunked = xr.open_dataset(url)
# What is the variable called?  Also check that dimensions and attributes look OK
unchunked

In [ ]:
# Check the total size of the dataset, and the details of the Runoff variable
print(f'Data is {unchunked.nbytes / 10**9} GB in size')
unchunked.Runoff

So our runoff data is about 1.5 GB, and already has a `_ChunkSizes` attribute.  What's going on here?

It turns out that this file has been compressed, and therefore [stored in chunks](https://www.unidata.ucar.edu/blogs/developer/entry/netcdf_compression) (because decompression is all-or-nothing, and this way "all" is a single chunk, not the whole file).  Chunking *also* [makes common access patterns much more efficient](https://www.unidata.ucar.edu/blogs/developer/entry/chunking_data_why_it_matters), if you are careful when [choosing the shape of your chunks](https://www.unidata.ucar.edu/blogs/developer/entry/chunking_data_choosing_shapes)  (links to blog posts by the NetCDF developers).

If you don't want to read those posts, take my word for it: you should always compute with chunks that are an integer multiple of the stored chunks, and aim for between 10MB and 100MB of data per chunk.  [Chunks may be choosen automatically in a future version of Xarray](https://github.com/pydata/xarray/issues/1440).

It's a good habit when accessing data from Thredds and critical when working on very large files, so here's how to do that in Xarray:

In [ ]:
monthly_runoff = xr.open_dataset(url, chunks=dict(time=30, lat=720, lon=720)).Runoff
monthly_runoff

The only difference from `unchunked.Runoff` above is that instead of a NumPy array (which actually exists in memory), Xarray is now using [a Dask array](http://xarray.pydata.org/en/stable/dask.html).  You can operate on Dask arrays in all the usual ways, but instead of calculating a result they determine *how* to calculate a result.  When you finally need the data, call `data.compute()` (or `data.plot()`, etc) - and then Dask will find the most efficient equivalent chunk-by-chunk process and execute that instead.  You can therefore work on any data that doesn't fit in memory, and put off choosing a subset until the last moment without any penalty.  The downside is that Dask operations are usually ten to twenty percent slower than the Numpy equivalent, so it's a good idea to load data into memory once it's small enough.

Of course, you can make plots from Xarray data backed by Dask in exactly the same way as usual:

In [ ]:
# You should be careful about plotting commands, as they will download
# all data in (eg) the most recent time-step.  OK in this case, but not always!
monthly_runoff.isel(time=-1).plot.imshow(robust=True)

In [ ]:
# And you can also calculate the mean, in the same way as usual.
last_mean = monthly_runoff.isel(time=-1).mean()
last_mean

Here, we haven't actually *computed* the mean yet - only worked out *how* to compute it.
To get the number, instead of a pending aggregation object, run:

In [ ]:
last_mean.compute()

Congratulations!  All of this code would work in exactly the same way for a many-terabyte dataset, where a single variable might be hundreds of gigabytes.  Best to analyse those on the supercomputer where they're stored though, because it might take months to load otherwise - that's why climate model outputs are "uploaded" as boxes of hard drives.

## Calculating temporal statistics

Time to get back to today's main topic: reporting and exporting data, first of all by producing a new version of the BOM graph with all countries.  If you noticed that our data is monthly and the graph is yearly, good work!  

Annual variance and monthly variance are very different things, so we will want to calculate the mean along the time dimension for months within each year.  This is a common form of operation, called "split-apply-combine", which is [built in to Pandas]() and therefore [also easy in Xarray](http://xarray.pydata.org/en/stable/groupby.html).  You can group by individual values along each coordinate dimension, or by 'bins' of coordinates.  A time dimension has special support for describing the groups as text, with `time.year` (or season, month, day, hour, minute, ...), because temporal grouping is such a common case.

In [ ]:
# We'll use the unchunked data here, because the lab computers aren't amazingly
# fast, the data fits in memory, and a tutorial spent debugging is no fun.
runoff = unchunked.Runoff.groupby('time.year').mean(dim='time')
runoff

In [ ]:
runoff.sel(year=2004).plot.imshow(robust=True)

Now that we have a nice data cube with annual mean runoff, we can calculate 
the temporal mean for all years, as well as the standard deviation in annual 
values. Subsequently, the Coefficient of Variation is defined as the ratio of 
standard deviation over mean, as follows:

In [ ]:
variation = runoff.std(dim='year') / runoff.mean(dim='year')
variation.plot.imshow(robust=True)

As you can see, Australia has high streamflow variability (that is, a 
high C.V. in annual average values), but so do many other places with low rainfall 
(arid regions and the poles). This does not answer the question as to what country 
wins the international streamflow variability competition, however. To work 
that you, we need to look at the values of C.V. in each of the countries. 


## Combining a polygon vector file with gridded data

To calculate national statistics by combining a gridded variable with a polygon 
vector file of country boundaries, there are at least three possible solutions:

- Export the grid from Python, and do the vector analysis entirely in ArcGIS
- Use ArcGIS to convert the vector layer to a grid, and do the remaining analysis in Python
- Use Python to convert the vector layer, and to do the remaining analysis.

You can of course substitute MatLab for Python, if you want to spend more money
on software, or substitute QGIS for ArcGIS if you'd rather spend less.

In order to stay with Python, we'll convert the vector layer to a grid.  You can find the
[instructions for ArcGIS here (pdf)](https://dl.dropboxusercontent.com/u/90979086/ENVS3019/tutorials/MatLab/Creating%20a%20query%20grid%20in%20ArcMap.pdf);
replacing "exported GeoTIFF from MatLab" with "exported NetCDF from Python"
(with `variation.to_netcdf('my_filename.nc')`) and using the NetCDF toolbox to 
read and write files.

If you are using Python at home - i.e. not on the ANU lab computers - I recommend
installing [Salem](https://salem.readthedocs.io/en/latest/installing.html) to read
Geotiff files into Xarray, and [GeoPandas](http://geopandas.org) to work with vector
data of all kinds.  Because we can't install them in our labs, here's a non-interactive
copy of the code I ran to do this:

```python
# We only want the metadata for each polygon, so we drop the geometry column
import geopandas
geopandas.read_file('TM_WORLD_BORDERS-0.3.shp').drop('geometry', axis=1).to_csv('data/countries.csv')
# If you export a GeoTIFF (as Albert did), here's how to convert it to NetCDF
# Salem can also convert shapefiles to a grid, if you're working entirely in Python!
import salem
salem.open_xr_dataset('CountryGrid2.tif').to_netcdf('data/CountryGrid.nc', encoding={'data': {'zlib': True}})
```

That really is all of my code - if you find the right tools for the job, most problems become much easier.  
Now let's open these files, or a version that you created in ArcGIS, in our notebook:

In [ ]:
cgrid = xr.open_dataarray('data/CountryGrid.nc')
cgrid

In [ ]:
# We'd better rename the X and Y coordinates to lat and lon,
# and ensure that they match our data for compatibility
cgrid = cgrid.rename(dict(x='lon', y='lat'))
cgrid.lat.values[:] = variation.lat
cgrid.lon.values[:] = variation.lon
cgrid.plot.imshow()

So we have a grid of unsigned 8-bit integers (0 to 255), but we don't know which number represents what country.  
Luckily, we also have metadata from the original shapefile:

In [ ]:
countries = pd.read_csv('data/countries.csv')
countries.head(5)

You can look up the meaning of each column on [the source website](http://thematicmapping.org/downloads/world_borders.php), but for our purposes the important bit is the name, lat, and lon.  Knowing the land area, population, country codes, and region is nice, but not part of this analysis!  

`Unnamed: 0` is a default name that Pandas gives to columns without a heading, and in this case represents the order that the polygons appeared in the shapefile.  Since this is the way that the grid was labelled, we should rename this something like `LABEL`.

It's usually a good idea to "clean" your data by dropping columns that you don't
want, any rows with missing data, and set the index to a meaningful (and unique!) identifying column (eg ISO2).
This is pretty quick, and can be done while loading the file if you're using a lot
of data in the same format.

In [ ]:
# In this case, we explictly keep the columns we want.
# See the geopandas example for how to drop one of many columns. 
countries['LABEL'] = countries['Unnamed: 0']
countries = countries[['ISO2', 'NAME', 'LAT', 'LON', 'LABEL']]
countries = countries.set_index('ISO2')
countries.head(5)

For example, we can select Australia (`AU`), and get the label.
Then we use `.where` to select the pixels where the query returns True,
and finish by plotting it.  Note that `drop=True` is an optional argument
to where, which clips the output grid to the extent of the data.

In [ ]:
label = countries.xs('AU').LABEL
variation.where(cgrid == label, drop=True).plot.imshow(robust=True)

We can then use a loop to do  this for all countries and take the mean, or use a 
[list comprehension](http://greenteapress.com/thinkpython2/html/thinkpython2020.html#sec224)
instead of an explicit `for` loop, and add the list as a new column in the dataframe.
Note that the mean of an Xarray dataarray is a zero-dimensional array, so I use `float()`
to convert it to a number.

In [ ]:
var_means = [float(variation.where(cgrid == label).mean()) for label in countries.LABEL]
countries['VARIATION'] = var_means
countries.head(10)

Hmm, looks like some of the islands are so small that there are no corresponding pixels in our grid!
Luckily, we can simply drop these values and move on:

In [ ]:
countries.dropna(inplace=True)
countries.head(2)

In [ ]:
countries.sort_values(by='VARIATION', ascending=False, inplace=True)
countries.head(20)

So there we have it, our international ranking. If we discount Antarctica 
it looks like Egypt is the 'winner', with a score more than double that of Australia! 
Oh well. Of course, the real winners are probably those countries that suffer 
the least interannual streamflow variability - congratulations to the Faroe 
Islands (see: `countries.tail(5)`).

Of course we can easily write this to a file with `countries.to_csv('rankings.csv')`,
which you could use to make a choropleth map in ArcGIS - or Python, for that matter.
However, it is hard to beat the publishing quality of a specialised 
GIS such as ArcMap, and it is very easy to join the CSV file written here to 
the country polygon file. Joins and relates were covered in the Social GIS tutorials 
(see page 8 of the pdf for gravity modelling tutorial 1, or follow 
[this link](https://wattlecourses.anu.edu.au/pluginfile.php/1346268/mod_page/content/14/Gravity_Model%20Lab_2017.pdf)).
It is recommended that you try that so you are familiar with the process.

In [ ]:
# Export the csv here if you want to try making a map


So, overall, does Australia have an unusually high interannual streamflow 
variability? It is obviously not unique in this way, but perhaps 
you could say that it has very high streamflow variability when not counting 
countries with very low runoff (perhaps they do not rely on river flows).


## Reusing code
We can calculate their streamflow easily, and this example demonstrates 
the benefit of scripting in Python rather than clicking away in a GIS.

In [ ]:
mean_runoff = runoff.mean(dim='year')
# Copy-pasted from the cell above, with mean_runoff instead of variation
var_means = [float(mean_runoff.where(cgrid == label).mean()) for label in countries.LABEL]
countries['RUNOFF'] = var_means
countries.head(5)

In [ ]:
# Let's test this hypothesis with a scatter plot!
# logx=True puts the x axis on a log scale, because our data is widely spread
countries.plot.scatter(x='RUNOFF', y='VARIATION', logx=True)

There is clearly a relationship, but how best to use this new data?  If we want to support a chosen conclusion,
we can filter the list to only includes countries with at least as much streamflow as Australia!

In [ ]:
# Another application of selecting with a boolean grid - one-dimensional this time
filtered = countries[countries.RUNOFF >= countries.RUNOFF.xs('AU')]
filtered.head(5)

In [ ]:
# We can also make a bar graph, with just the first ten rows so we can read the labels
filtered.head(10).plot.bar(x='NAME', y='VARIATION')

Now, all we need to do is find reasons to disqualify Greenland (easy - 
[it's not really a country](https://www.google.com.au/search?q=is+greenland+a+country))
or Pakistan (hmm..harder). Perhaps we can win this competition yet, or at least be runner-up.
It shows you the statistical power of cherry-picking.


## Summary and Research Ideas
This Tutorial focused on calculating temporal and spatial statistics and combining 
a query grid derived from polygon data to calculate those statistics for different 
regions separately. These analyse are all helpful to summarise a large volume 
of data into a smaller volume and subsequently to visualise, explore and communicate 
the summary results (e.g., thematic maps, ranking tables and data plots).

Here, we tested whether Australia really has such a high interannual streamflow 
variability, and found that it is high but not uniquely so globally. We could 
illustrate this with a thematic map, bar graph, ranking table or scatterplot -
made in Python, Excel or ArcMap, depending on preference.

You could adapt the code shown here to do a similar type of summary but 
for different environmental variables and region types. For example, methods 
and MatLab code very similar to that used here were used to calculate data summaries 
by state, catchment, statistical area etc that you can find as thematic maps, 
bar and line graphs and pie charts in Australia's Environment Explorer.
Transforming spatial data into tabulated data like that is often 
the first step in reporting in, e.g., environmental accounts.

Of course, you can also combine regional summaries with any of the previous 
tutorials. For example, you could calculate the regional summaries of grid-based 
trend analysis or the other way around: you could perform trend analysis on 
time series spatially averaged by region. Note that the results will not be 
identical or even necessarily similar: the spatial mean of a temporal standard 
deviation can be very different from the temporal standard deviation of a spatial mean.


## Finally
This was the last tutorial. By now, I hope you will have a much better idea 
of how remote sensing data is analysed, summarised and visualised. You can now 
legitimately say that you have some experience in those areas. 

As with most learned skills, getting some instruction is only the start 
to developing strong skills. The rest is in ongoing practice and self-education 
(in writing these tutorials both Albert and Zac learned several new things!). The key to ongoing 
self-education is knowing how to find answers. We are around during the tutorials 
and by email to help you with that. After this course, key tools are the Python 
`help` function, reading the documentation for whatever package you are using, 
and also just googling your question (try, for example "python xarray time average"). 

Hopefully, you are now convinced that programming, while not always easy 
and sometimes just painful, empowers us to do things that would be otherwise 
impossible, and that it can be a major save on time, energy and frustration. 
Who knows, you may have even found it somewhat satisfying or even fun. Worse 
again, you might even like to develop your programming skills further? 

If so, then it was great we could show you a few tricks, and from here 
on: happy programming!

Zac Hatfield-Dodds, May 2017  
[zac.hatfield.dodds@anu.edu.au](mailto:zac.hatfield.dodds@anu.edu.au?subject=Python%20remote%20sensing%20tutorials)